# DunnHumby Recommender System 

Update February 2022 --

In the process of eventually deploying an app to `Heroku`, I've come a long way in terms of organizing my ETL structures. I'm grateful for having spent time learning more about coding in Python. This class is far too expensive, computationally, to run on a remote server. 

That limitation has forced me to consider alternatives -- and structure my 'business outcomes' in a more readable way. I'm going to go back and create several spectra of class labels; and then compare modelling results for each, such that I can find the households most similar to one another in certain areas -- this process in the end may involve more market basket analysis; fpgrowth; and association rules analysis. I'm figuring it out, albeit slowly. 

In the end I hope to be able to provide several core recommendation tables; for each of 3-5 class labels for households in the data. Moreover; to be able to generate those class labels using new data that might become available.


Because of the complexity of this dataset, and my weak background in statistics, I opted to showcase a Python class structure capable of generating recommendations for any given household based on their previous purchases. 

My thinking is that a system like this would be able to effectively identify profitable spending patterns by a household -- those associated products which influence the customer to spend more or purchase more frequently. 

For example, if our competitor pricing research found that we should target a specific product in order to vie for market share; **the recommender system below would be able to identify what other products might do well alongside the 'targeted' item in a marketing campaign to that customer. With multiple reasons to come to the store, the customer might be more inclined to make the trip, or to spend more if they do**. 

I recognize that in a brick-and-mortar store, having live-updating recommendations based on items in basket is not necessarily as useful as it would be in an online shop -- however, I believe there is value to be offered by using a system like this.

Data Pipeline
---

The concept (and practice!) of molding data into a 'pipeline' is a deep one -- the 'correct answer' has to be negotiated based on current data infrastructure, available resources and future scope.

An effective data pipeline would:

- generate or otherwise acquire data
- ensure the data's cleanliness and integrity
- seamlessly add the data to an existing data pool
- access that data as needed, for analysis and modelling
- surface that analysis and modelling results

For the scope of this project, we don't have much of a chance to acquire more data. But, if we assume that any other data would be in the same format as the data we've already received (big leap), then we've already made some work on creating a pipeline for future data.

**pipeline thus far**:
- commodity breakdown/sales analysis
- adding datetime
- data truncation
- customer segmentation (RFM score)


The pipeline is important, as it allows us to quickly identify changing patterns in customer purchase behaviour in the case of live or newly acquired data in the same format.

Adding datetime is a 'housekeeping' issue which makes things easier on our end. 

Customer segmentation, however, provides some valuable insight about any given customer (household) in our data. 

By segmenting the customers in various ways, we can create effective recommendations for a given subset of our customers -- for example, the most profitable ones. 

In [2]:
#import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16,6)
import datetime

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

import dtcj


ModuleNotFoundError: No module named 'mlxtend'

Forming `merged` from `product.csv` and `transaction_data.csv`
---

# Recommender System for Households
---

Using FPGrowth (apriori and association rules)

Below I've implemented a rudimentary (minimum viable product?) Recommender System class.

There is so much left for me to learn about how a class like this might be integrated into 'production'.


In [ ]:
#import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16,6)
import datetime

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

import dtcj

class RecommenderSystem:
    '''
    ## hh_key :  the household_key
    ## df : the transactions df; 
    ## column : the column in df to be used for MBA ('COMMODITY_DESC, DEPARTMENT, SUB_COMMODITY_DESC')
    ## max_len :  max length of any antecedent/consequent chains in support_table
    ## support_threshold : minimum 'support' threshold to generate fpgrowth
    
    ## metric : the association rules metric to maximize
    ## assoc_threshold : the association rules threshold, given the metric.
    '''
        ## Instantiate Class

    def __init__(self, 
                 hh_key,   ### FOR HOUSEHOLDS#!### 
                 df=merged, 
                 column='COMMODITY_DESC', 
                    max_len=None, ### CONSIDER REDUCING THIS VALUE FOR SIMPLICITY ###
                 support_threshold=0.05, ### WITH DATA OF FIXED SIZE, NOT A CONCERN? ###
                metric='confidence', 
                 assoc_threshold=0.8,
                ):
                                    #TODO: implement intelligent thresholds
        self.hh = hh_key
        self.metric = metric
        self.assoc_threshold=assoc_threshold
        self.column = column
        self.support_threshold = support_threshold
        self.df = df[df['household_key'] == self.hh] # self.df is transactions for this customer only
        self.max_len = max_len
        self.product_lists = []

        # create support table upon instantiation
        
        self.get_support_table()
     
    
    ### Support Table Function ###
    # uses fpgrowth to generate a support table
    def get_support_table(self):
        '''Return the support table for `BASKET_ID`s using `column` as product lists
        Note: 'BASKET_ID' is hardcoded...
        
        '''
        # create product lists for each basket                                   
        self.product_lists = self.df.groupby('BASKET_ID')[self.column].apply(list) # apply list constructor
 
        # dummy encoding...
        te = TransactionEncoder()
        te_fit = te.fit_transform(self.product_lists.values, sparse=True) # encode each 
        te_df = pd.DataFrame.sparse.from_spmatrix(te_fit, columns=[str(i) for i in te.columns_])
      
        # fpgrowth table
        frequent_itemsets = fpgrowth(te_df, 
                                    min_support=self.support_threshold, #can alter self.support_threshold
                                    use_colnames=True, 
#                                     verbose=True, 
                                    max_len=self.max_len,   # can alter self.max_len here.
                                    #, low_memory=True,                                   
                                    )
        # adding a length column for posterity and filtering
        frequent_itemsets['size'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
        
        # save variable for reference...
        self.support_table = frequent_itemsets

    
    @property
    def assoc_table(self):
        '''change self.metric, self.assoc_threshold to rank differently'''
        ##  calling association rules on our support table
        rules = association_rules(self.support_table, metric=self.metric, min_threshold=self.assoc_threshold)
        rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
        return rules
    
    def recommend(self, prev_purchases:list, howmany=5):
        '''meat and bones of the recommender system...
        accepts:
            prev_purchases: a list of previously purchased items
            howmany: (int) how many recommendations you want
            
        returns:
            a series consisting of the top 5 results given the self.metric value.
            '''
        search_terms = list(prev_purchases) # handles frozensets?
        # apply list to 'antecedent' in self.assoc_table
        search_series = pd.Series(self.assoc_table['antecedents'].apply(list)) 
        
        print(f'Searching for {search_terms}...')
        indexes_of_matches = []

        # for each antecedent chain...
        for item in search_terms:
            # iterate through the list of "antecedent" rows **search_series**
            for idx, val in search_series.iteritems():
                if item in val: # if the item is in the row (list of antecedents)..
                    indexes_of_matches.append(idx)

        rules = self.assoc_table.loc[indexes_of_matches]


        # RETURN TOP 5 LIFT CONSEQUENTS
        return rules.sort_values(self.metric, ascending=False)[:howmany]['consequents']

# Looping Recommendations to form new Table

In [ ]:
merged = dtcj.load_merged()

In [ ]:
#import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16,6)
import datetime

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

import dtcj


class NewRecommenderSystem:
    '''
    ## hh_key :  the household_key
    ## df : the transactions df; 
    ## column : the column in df to be used for MBA ('COMMODITY_DESC, DEPARTMENT, SUB_COMMODITY_DESC')
    ## max_len :  max length of any antecedent/consequent chains in support_table
    ## support_threshold : minimum 'support' threshold to generate fpgrowth
    
    ## metric : the association rules metric to maximize
    ## assoc_threshold : the association rules threshold, given the metric.


    
        TODO: 
        Move chosen metric to the 'recommendation' function; 
            - allow for recommendations based on the 'strength' of the given metric against the overall mean and the metric's other values; as well as across other metrics?
              
                  '''
        ## Instantiate Class

    def __init__(self, 
                 hh_key,   ### FOR HOUSEHOLDS#!### 
                 df, 
                 column='COMMODITY_DESC', 
                    max_len=None, ### CONSIDER REDUCING THIS VALUE FOR SIMPLICITY ###
                 support_threshold=0.05, ### WITH DATA OF FIXED SIZE, NOT A CONCERN? ###
                metric='confidence', 
                 assoc_threshold=0.8,
                ):
                                    #TODO: implement intelligent thresholds
        self.hh = hh_key
        self.metric = metric
        self.assoc_threshold=assoc_threshold
        self.column = column
        self.support_threshold = support_threshold
        self.df = df[df['household_key'] == self.hh] # self.df is transactions for this customer only
        self.max_len = max_len
        self.product_lists = []

        # create support table upon instantiation
        
        self.get_support_table()
     
    
    ### Support Table Function ###
    def get_support_table(self):
        '''Return the support table for all `BASKET_ID`s using `column` as product lists...?
        This should use the support table for only the individual's purchases, no?
        Note: 'BASKET_ID' is hardcoded...
        
        '''

        # create product lists for each basket   
        self.product_lists = self.df[self.df['household_key'] == self.hh].groupby('BASKET_ID')[self.column].apply(list) 

        # dummy encoding...
        te = TransactionEncoder()
        te_fit = te.fit_transform(self.product_lists.values, sparse=True) # encode each 
        te_df = pd.DataFrame.sparse.from_spmatrix(te_fit, columns=[str(i) for i in te.columns_])
      
        # fpgrowth table
        frequent_itemsets = fpgrowth(te_df, 
                                    min_support=self.support_threshold, #can alter self.support_threshold
                                    use_colnames=True, 
#                                     verbose=True, 
                                    max_len=self.max_len,   # can alter self.max_len here.
                                    #, low_memory=True,                                   
                                    )
        # adding a length column for posterity and filtering
        frequent_itemsets['size'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
        
        # save variable for reference...
        self.support_table = frequent_itemsets

    
    @property
    def assoc_table(self):
        '''change self.metric, self.assoc_threshold to rank differently'''
        ##  calling association rules on our support table
        rules = association_rules(self.support_table, metric=self.metric, min_threshold=self.assoc_threshold)
        rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
        return rules
    
    def recommend(self, prev_purchases:list, howmany=5):
        '''meat and bones of the recommender system...
        accepts:
            prev_purchases: a list of previously purchased items
            howmany: (int) how many recommendations you want
            
        returns:
            a series consisting of the top 5 results given the self.metric value.
            '''
        search_terms = list(prev_purchases) # handles frozensets?
        # apply list to 'antecedent' in self.assoc_table
        search_series = pd.Series(self.assoc_table['antecedents'].apply(list)) 
        
        print(f'Searching for {search_terms}...')
        indexes_of_matches = []

        # this generates the full table of recommendations, no matter how small 'howmany' is.... crazy

        # for each antecedent chain...
        for item in search_terms:
            # iterate through the list of "antecedent" rows **search_series**
            for idx, val in search_series.iteritems():
                if item in val: # if the item is in the row (list of antecedents)..
                    indexes_of_matches.append(idx)

        rules = self.assoc_table.loc[indexes_of_matches]


        # RETURN TOP 5 LIFT CONSEQUENTS
        return rules.sort_values(self.metric, ascending=False)[:howmany]['consequents']

In [ ]:
data = dict()
# basic most-frequent heuristic
# generate association rules based on the items most-frequently purchased

for hh in merged['household_key'].unique()[:5]:
    prev_purchases = merged[merged['household_key']==hh]['COMMODITY_DESC'].value_counts()[:10].keys()
    #print(prev_purchases)
    instance = NewRecommenderSystem(df=merged, hh_key=hh, metric='lift')
    output = instance.recommend(prev_purchases=prev_purchases).values
    print(output)

    recommendations = list(frozenset().union(*output))
    #print(recommendations)

    data[hh] = recommendations


Searching for ['CANDY - CHECKLANE', 'SOFT DRINKS', 'CANNED JUICES', 'SOUP', 'BAG SNACKS', 'BAKED SWEET GOODS', 'CRACKERS/MISC BKD FD', 'FLUID MILK PRODUCTS', 'BEEF', 'FROZEN PIZZA']...
[frozenset({'CRACKERS/MISC BKD FD', 'BAKING MIXES'})
 frozenset({'BAKED SWEET GOODS', 'FLUID MILK PRODUCTS'})
 frozenset({'BAKED SWEET GOODS', 'FLUID MILK PRODUCTS'})
 frozenset({'REFRGRATD JUICES/DRNKS', 'CRACKERS/MISC BKD FD'})
 frozenset({'REFRGRATD JUICES/DRNKS', 'BAKED SWEET GOODS', 'CRACKERS/MISC BKD FD'})]
Searching for ['FLUID MILK PRODUCTS', 'VEGETABLES - ALL OTHERS', 'WATER - CARBONATED/FLVRD DRINK', 'APPLES', 'TOMATOES', 'CRACKERS/MISC BKD FD', 'BREAD', 'BROCCOLI/CAULIFLOWER', 'COLD CEREAL', 'SQUASH']...
[frozenset({'BROCCOLI/CAULIFLOWER'})
 frozenset({'PEARS', 'FLUID MILK PRODUCTS'}) frozenset({'PEARS'})
 frozenset({'PEARS'}) frozenset({'EGGS'})]
Searching for ['CHEESE', 'BAKED BREAD/BUNS/ROLLS', 'SOFT DRINKS', 'LUNCHMEAT', 'FLUID MILK PRODUCTS', 'BEEF', 'CANDY - PACKAGED', 'BERRIES', 'FROZEN

In [ ]:
    #print(prev_purchases)
from collections import Counter
Counter([len(x) for x in data.values()])
# recommendations of equal length for each household



In [ ]:

data[2319]

55                  (CANNED JUICES)
546                          (BEEF)
451          (CRACKERS/MISC BKD FD)
453       (BAKED SWEET GOODS, BEEF)
455    (CRACKERS/MISC BKD FD, BEEF)
Name: consequents, dtype: object

May wish to look at alternative interpretations of the association_Rules table in the class, in order to generate recommendations. 


## Demonstration
---

For `household_key == 2`, looking at 'confidence':

In [ ]:
hh2 = RecommenderSystem(hh_key=2, metric='confidence', assoc_threshold=0.8)#, max_len=2)

print(hh2.metric)
hh2.recommend(['APPLES'])
# returns the top 5 'confidence' of all purchases with 'APPLES' antecedent for hh_key=2

# these are the 5 products most often correlated with the purchase of 'APPLES'.

confidence
Searching for ['APPLES']...


174694                              (EGGS)
179432    (BAKED BREAD/BUNS/ROLLS, YOGURT)
179421                        (MARGARINES)
179422            (BAKED BREAD/BUNS/ROLLS)
179424                            (YOGURT)
Name: consequents, dtype: object

In [ ]:
hh2.df['COMMODITY_DESC'].value_counts()

SOFT DRINKS                     48
FLUID MILK PRODUCTS             28
BAKED BREAD/BUNS/ROLLS          22
BAKING MIXES                    19
DOG FOODS                       17
                                ..
HOT CEREAL                       1
FRZN MEAT/MEAT DINNERS           1
BREAKFAST SAUSAGE/SANDWICHES     1
FIRST AID PRODUCTS               1
CANDY - PACKAGED                 1
Name: COMMODITY_DESC, Length: 140, dtype: int64

In [ ]:
len(hh2.product_lists)

45

Looking at 'lift'

In [ ]:
hh2 = RecommenderSystem(hh_key=2, metric='lift', assoc_threshold=0.8, max_len=2)
print(hh2.metric)
hh2.recommend(['APPLES'])
# returns the top 5 'lift' of all purchases with 'APPLES' antecedent.
# when the customer is purchasing apples, these products are often purchased (when they might not have been otherwise) 

lift
Searching for ['APPLES']...


556                  (YOGURT)
513    (FRUIT - SHELF STABLE)
483    (CRACKERS/MISC BKD FD)
349              (MARGARINES)
264              (BAG SNACKS)
Name: consequents, dtype: object

Looking at 'conviction':

In [ ]:
hh2.metric='conviction'
hh2.recommend(['APPLES']) 

Searching for ['APPLES']...


243       (FLUID MILK PRODUCTS)
249               (SOFT DRINKS)
250    (BAKED BREAD/BUNS/ROLLS)
264                (BAG SNACKS)
267         (BAKED SWEET GOODS)
Name: consequents, dtype: object

Interpretation
---

For multi-item antecedent chains (items in 'basket'), this recommender searches for recommendations for each item independently, then concatenates the full table of results and sorts by the metric currently selected in the class .metric attribute.

So many things to consider, with a technology like this. Does it make sense to filter more stringently, and only look for chains of antecedents precisely equal to the search term? What about finding similar products using something like word2vec or other NLP? How do we change or vary our recommendations, so that customers aren't constantly seeing the same recommendations?

We could use cluster labels, RFM score, demographics to populate a 'similar customer' association rules table, and make recommendations from there. A basic approach would be to recommend based on recently purchased items; or to reverse-engineer this process and recommend 'trigger' products which we might want to be selling to them.

We're getting redundant values in our consequent chains when parsing 5 straight from the top of the `confidence` metric. What about returning the results from a blend of metrics; another option would be to add another layer of logic to prevent redundant recommendations. (create a 'set', and don't stop filling it until it reaches 5 items). 

One issue about having an antecedent 'chain' instead of a single result: the recommendations won't always be obviously related. This could be frustrating for a customer. 


Run the recommender on all of the purchases by a household using the support table of similarly-labeled customers.

Alternative Methods of Recommendation
---


- Instead of using the fpgrowth table for each household proper; use the Cluster label (or RFM score group) to generate a 'similar-customers' fpgrowth table; call .recommend() using that table, for the most recent purchases by the household.

- Search a customer's history to see their 'DEPARTMENT' or 'COMMODITY' spending vs the average? 

- Get the rows for each top ranked value in a given support table.. #TODO: deal with ties, inf 

- Use NLP or regex to find similar items and make the search terms more robust.

## Second Example

Expanding on the functionality
---


By altering the last few lines of the recommend() function, we could instead return a `set` of items from the top of our results; or add several recommender metrics together and return the set of all items. for example:

In [ ]:
class RecommenderSystemSET:
    '''
    ## hh_key :  the household_key
    ## df : the transactions df; 
    ## column : the column in df to be used for MBA ('COMMODITY_DESC, DEPARTMENT, SUB_COMMODITY_DESC')
    ## max_len :  max length of any antecedent/consequent chains in support_table
    ## support_threshold : minimum 'support' threshold to generate fpgrowth
    
    ## metric : the association rules metric to maximize
    ## assoc_threshold : the association rules threshold, given the metric.
    '''
        ## Instantiate Class

    def __init__(self, 
                 hh_key,   ### FOR HOUSEHOLDS#!### 
                 df=merged, 
                 column='COMMODITY_DESC', 
                    max_len=None, ### CONSIDER REDUCING THIS VALUE FOR SIMPLICITY ###
                 support_threshold=0.05, ### WITH DATA OF FIXED SIZE, NOT A CONCERN? ###
                metric='confidence', 
                 assoc_threshold=0.8,
                ):
                                    #TODO: implement intelligent thresholds
        self.hh = hh_key
        self.metric = metric
        self.assoc_threshold=assoc_threshold
        self.column = column
        self.support_threshold = support_threshold
        self.df = df[df['household_key'] == self.hh] # self.df is transactions for this customer only
        self.max_len = max_len
        self.product_lists = []

        # create support table upon instantiation
        
        self.get_support_table()
     
    
    ### Support Table Function ###
    
    # uses fpgrowth to generate a support table
    
    def get_support_table(self):
        '''Return the support table for `BASKET_ID`s using `column` as product lists
        Note: 'BASKET_ID' is hardcoded...
        
        '''
        # create product lists for each basket                                   
        self.product_lists = self.df.groupby('BASKET_ID')[self.column].apply(list) # apply list constructor
 
        # dummy encoding...
        te = TransactionEncoder()
        te_fit = te.fit_transform(self.product_lists.values, sparse=True) # encode each 
        te_df = pd.DataFrame.sparse.from_spmatrix(te_fit, columns=[str(i) for i in te.columns_])
      
        # fpgrowth table
        frequent_itemsets = fpgrowth(te_df, 
                                    min_support=self.support_threshold, #can alter self.support_threshold
                                    use_colnames=True, 
#                                     verbose=True, 
                                    max_len=self.max_len,   # can alter self.max_len here.
                                    #, low_memory=True,                                   
                                    )
        # adding a length column for posterity and filtering
        frequent_itemsets['size'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
        
        # save variable for reference...
        self.support_table = frequent_itemsets

    
    @property
    def assoc_table(self):
        '''change self.metric, self.assoc_threshold to rank differently'''
        ##  calling association rules on our support table
        rules = association_rules(self.support_table, metric=self.metric, min_threshold=self.assoc_threshold)
        rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
        return rules
    
    
    def recommend(self, prev_purchases:list, howmany=5):
        '''Edited Function
            '''

        search_terms = list(prev_purchases) # handles frozensets?
        # apply list to 'antecedent' in self.assoc_table
        search_series = pd.Series(self.assoc_table['antecedents'].apply(list)) 

        print(f'Searching for {search_terms}...')
        indexes_of_matches = []

        # for each antecedent chain...
        for item in search_terms:
            # iterate through the list of "antecedent" rows **search_series**
            for idx, val in search_series.iteritems():
                if item in val: # if the item is in the row (list of antecedents)..
                    indexes_of_matches.append(idx)

        rules = self.assoc_table.loc[indexes_of_matches]

        # RETURN TOP 5 LIFT CONSEQUENTS
        results = []
        
        
        # TODO: check if the tables are populating correctly given only self.metric...
        for metric in ['confidence', 'conviction', 'lift', 'leverage', 'support']:
            results.extend(list(rules.sort_values(metric, ascending=False)[:howmany]['consequents']))

        set_results = []
        for x in results:
            set_results.extend(list(x))
        #       
        return set(set_results)


In [ ]:
hh2 = RecommenderSystemSET(2)

In [ ]:
set(merged[merged['household_key']==2]['COMMODITY_DESC']) # all previous purchases

{'ADULT INCONTINENCE',
 'AIR CARE',
 'ANALGESICS',
 'APPLES',
 'BABY FOODS',
 'BACON',
 'BAG SNACKS',
 'BAKED BREAD/BUNS/ROLLS',
 'BAKED SWEET GOODS',
 'BAKING',
 'BAKING MIXES',
 'BAKING NEEDS',
 'BATH',
 'BATH TISSUES',
 'BATTERIES',
 'BEEF',
 'BEERS/ALES',
 'BERRIES',
 'BLEACH',
 'BREAD',
 'BREAKFAST SAUSAGE/SANDWICHES',
 'BREAKFAST SWEETS',
 'BROCCOLI/CAULIFLOWER',
 'BROOMS AND MOPS',
 'CANDY - CHECKLANE',
 'CANDY - PACKAGED',
 'CANNED JUICES',
 'CARROTS',
 'CHEESE',
 'CHEESES',
 'CHICKEN',
 'CHRISTMAS  SEASONAL',
 'CIGARETTES',
 'CITRUS',
 'COFFEE',
 'COFFEE FILTERS',
 'COLD CEREAL',
 'CONDIMENTS/SAUCES',
 'CONVENIENT BRKFST/WHLSM SNACKS',
 'COOKIES/CONES',
 'COOKWARE & BAKEWARE',
 'CORN',
 'COUPON/MISC ITEMS',
 'CRACKERS/MISC BKD FD',
 'DELI MEATS',
 'DEODORANTS',
 'DINNER MXS:DRY',
 'DISHWASH DETERGENTS',
 'DISPOSIBLE FOILWARE',
 'DOG FOODS',
 'DRIED FRUIT',
 'DRY BN/VEG/POTATO/RICE',
 'DRY MIX DESSERTS',
 'DRY NOODLES/PASTA',
 'EGGS',
 'ELECTRICAL SUPPPLIES',
 'FACIAL TISS/DNR 

In [ ]:
# most frequent????

In [ ]:
most_recent = list(merged[merged['household_key']==2].sort_values('datetime')['COMMODITY_DESC'])[-5:]
most_recent

['PAPER HOUSEWARES',
 'SOFT DRINKS',
 'CANNED JUICES',
 'BREAD',
 'PET CARE SUPPLIES']

In [ ]:
hh2.metric = 'lift'

In [ ]:
hh2.recommend(most_recent)

Searching for ['PAPER HOUSEWARES', 'SOFT DRINKS', 'CANNED JUICES', 'BREAD', 'PET CARE SUPPLIES']...


{'BAKED BREAD/BUNS/ROLLS',
 'BAKING MIXES',
 'BAKING NEEDS',
 'BEEF',
 'CANNED JUICES',
 'COOKIES/CONES',
 'DEODORANTS',
 'DOG FOODS',
 'FLUID MILK PRODUCTS',
 'PET CARE SUPPLIES',
 'POTATOES',
 'SHORTENING/OIL',
 'SOFT DRINKS',
 'SUGARS/SWEETNERS',
 'VEGETABLES - SHELF STABLE',
 'YOGURT'}

In [ ]:
hh2.recommend(['BAKED BREAD/BUNS/ROLLS'])

Searching for ['BAKED BREAD/BUNS/ROLLS']...


{'BAKED SWEET GOODS',
 'BAKING MIXES',
 'BAKING NEEDS',
 'BEEF',
 'CANNED JUICES',
 'COOKIES/CONES',
 'FLUID MILK PRODUCTS',
 'MARGARINES',
 'ONIONS',
 'PET CARE SUPPLIES',
 'POTATOES',
 'SOFT DRINKS',
 'VEGETABLES - SHELF STABLE',
 'YOGURT'}

In [ ]:
hh5 = RecommenderSystem(hh_key=5, support_threshold=0.1) # consider opening this range further
hh5.metric='confidence'
hh5.assoc_threshold=0.2 # very low. 

hh5.assoc_table

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(SALADS/DIPS),(BAG SNACKS),0.166667,0.166667,0.111111,0.666667,4.0,0.083333,2.5,1
1,(BAG SNACKS),(SALADS/DIPS),0.166667,0.166667,0.111111,0.666667,4.0,0.083333,2.5,1


# Association Rules Metrics 

There is a ton of information that can be gathered by this sort of analysis;

By calculating the 'frequency of occurrence' (`support`) for each item (in this case, `COMMODITY_DESC`) as such, we can then perform calculations to determine **if the presence of one product in the purchase basket** has affected the chance that **another product will be purchased** ; by how much (`lift` factor); and how strong the association is (`confidence` proportion).

Below we can see some of the calculations quickly using the `association_rules` function:


For example, one top-rated association in the `lift` category from the table above is [butter and pasta sauce] -> [dry noodles and cheese]. This means that the chance of having dry noodles AND cheese in your basket -- if you have butter AND pasta sauce *already* -- is increased by a factor of almost 16 times compared to the original `supports` of those chains of items independent of one another. 

If instead we sort the table by `confidence`, we can look at the proportion of co-occurence the `antecedent` (chain) exists on it's own.

We create this table below using a threshold for the `confidence` metric; calculated as: how many baskets does the `consequent` appear in **with** the `antecedent`, divided by the total number of baskets in which the `antecedent` is present.

A confidence score approaching 1 reflects that the `consequent` is found in the `antecedent`'s baskets
approaching --> 100% of the time. 

Below we can see the highest-`confidence` relationships from the the purchases in our data, and the categories in `COMMODITY_DESC`.

In [ ]:
hh2.assoc_table

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(BAKED BREAD/BUNS/ROLLS),(SOFT DRINKS),0.355556,0.555556,0.288889,0.812500,1.462500,0.091358,2.370370,1
1,"(SOFT DRINKS, BAKED BREAD/BUNS/ROLLS)",(FLUID MILK PRODUCTS),0.288889,0.444444,0.244444,0.846154,1.903846,0.116049,3.611111,2
2,"(FLUID MILK PRODUCTS, BAKED BREAD/BUNS/ROLLS)",(SOFT DRINKS),0.266667,0.555556,0.244444,0.916667,1.650000,0.096296,5.333333,2
3,(TOMATOES),(BAKED BREAD/BUNS/ROLLS),0.111111,0.355556,0.111111,1.000000,2.812500,0.071605,inf,1
4,"(SOFT DRINKS, TOMATOES)",(BAKED BREAD/BUNS/ROLLS),0.066667,0.355556,0.066667,1.000000,2.812500,0.042963,inf,2
...,...,...,...,...,...,...,...,...,...,...
338935,"(DRY NOODLES/PASTA, DOG FOODS)","(SOFT DRINKS, DEODORANTS, BAKED BREAD/BUNS/ROL...",0.066667,0.066667,0.066667,1.000000,15.000000,0.062222,inf,2
338936,"(PASTA SAUCE, DOG FOODS)","(SOFT DRINKS, DEODORANTS, BAKED BREAD/BUNS/ROL...",0.066667,0.088889,0.066667,1.000000,11.250000,0.060741,inf,2
338937,"(DRY NOODLES/PASTA, PASTA SAUCE)","(SOFT DRINKS, DEODORANTS, BAKED BREAD/BUNS/ROL...",0.066667,0.111111,0.066667,1.000000,9.000000,0.059259,inf,2
338938,"(FLUID MILK PRODUCTS, PASTA SAUCE)","(SOFT DRINKS, DEODORANTS, BAKED BREAD/BUNS/ROL...",0.066667,0.066667,0.066667,1.000000,15.000000,0.062222,inf,2


In [ ]:
hh2.support_table

,support,itemsets,size
0,0.555556,(SOFT DRINKS),1
1,0.355556,(BAKED BREAD/BUNS/ROLLS),1
2,0.111111,(TOMATOES),1
3,0.111111,(STONE FRUIT),1
4,0.111111,(MILK BY-PRODUCTS),1
...,...,...,...
9726,0.066667,"(SOFT DRINKS, DEODORANTS, DOG FOODS, DRY NOODL...",6
9727,0.066667,"(SOFT DRINKS, DEODORANTS, BAKED BREAD/BUNS/ROL...",6
9728,0.066667,"(SOFT DRINKS, BAKED BREAD/BUNS/ROLLS, DOG FOOD...",6
9729,0.066667,"(SOFT DRINKS, DEODORANTS, BAKED BREAD/BUNS/ROL...",6


In [ ]:
hh2 = RecommenderSystem(2)
hh2.metric = 'confidence'
hh2.assoc_table.sort_values(hh2.metric, ascending=False).head(12)
#This table changes slightly without max_len=4

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
169470,"(BAKING MIXES, SOFT DRINKS, BAKED BREAD/BUNS/R...","(VEGETABLES - SHELF STABLE, BEEF, BAKING NEEDS)",0.066667,0.088889,0.066667,1.0,11.25,0.060741,inf,6
225583,"(BAKING MIXES, CANNED JUICES, FLUID MILK PRODU...","(BEEF, YOGURT, SOFT DRINKS, COOKIES/CONES, BAK...",0.066667,0.066667,0.066667,1.0,15.00,0.062222,inf,3
225581,"(SOFT DRINKS, BAKING MIXES, CANNED JUICES)","(BEEF, YOGURT, COOKIES/CONES, FLUID MILK PRODU...",0.066667,0.066667,0.066667,1.0,15.00,0.062222,inf,3
225580,"(BAKING MIXES, BAKING NEEDS, YOGURT)","(BEEF, SOFT DRINKS, COOKIES/CONES, CANNED JUIC...",0.066667,0.066667,0.066667,1.0,15.00,0.062222,inf,3
225579,"(BAKING MIXES, FLUID MILK PRODUCTS, YOGURT)","(BEEF, SOFT DRINKS, COOKIES/CONES, CANNED JUIC...",0.066667,0.066667,0.066667,1.0,15.00,0.062222,inf,3
225578,"(BAKING MIXES, CANNED JUICES, YOGURT)","(BEEF, SOFT DRINKS, COOKIES/CONES, FLUID MILK ...",0.066667,0.088889,0.066667,1.0,11.25,0.060741,inf,3
225577,"(COOKIES/CONES, BAKING MIXES, YOGURT)","(BEEF, SOFT DRINKS, CANNED JUICES, FLUID MILK ...",0.066667,0.066667,0.066667,1.0,15.00,0.062222,inf,3
225576,"(SOFT DRINKS, BAKING MIXES, YOGURT)","(BEEF, COOKIES/CONES, CANNED JUICES, FLUID MIL...",0.066667,0.066667,0.066667,1.0,15.00,0.062222,inf,3
225575,"(BAKING MIXES, CANNED JUICES, BEEF)","(YOGURT, SOFT DRINKS, COOKIES/CONES, FLUID MIL...",0.066667,0.066667,0.066667,1.0,15.00,0.062222,inf,3
225574,"(BAKING MIXES, BEEF, YOGURT)","(SOFT DRINKS, COOKIES/CONES, CANNED JUICES, FL...",0.066667,0.066667,0.066667,1.0,15.00,0.062222,inf,3
